In [1]:
from geopy.geocoders import Nominatim

# Initialize the geocoder
geolocator = Nominatim(user_agent="nyc_uber_geocoder")

# Function to reverse geocode a single pair of coordinates
def reverse_geocode(lat, lon):
    location = geolocator.reverse((lat, lon), exactly_one=True)
    return location.address if location else None

latitude = 40.748817
longitude = -73.985428
location = reverse_geocode(latitude, longitude)
print(location)


Naturalizer, 7, West 34th Street, Midtown South, Manhattan, New York County, City of New York, New York, 10001, United States


In [2]:
import pandas as pd

# Sample DataFrame
data = {
    'pickup_latitude': [40.748817, 40.712776],
    'pickup_longitude': [-73.985428, -74.005974],
    'dropoff_latitude': [40.730610, 40.706001],
    'dropoff_longitude': [-73.935242, -74.008819],
}
df = pd.DataFrame(data)

# Apply reverse geocoding to each row
df['pickup_location'] = df.apply(lambda row: reverse_geocode(row['pickup_latitude'], row['pickup_longitude']), axis=1)
df['dropoff_location'] = df.apply(lambda row: reverse_geocode(row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

print(df[['pickup_location', 'dropoff_location']])

                                     pickup_location  \
0  Naturalizer, 7, West 34th Street, Midtown Sout...   
1  New York City Hall, 260, Broadway, Lower Manha...   

                                    dropoff_location  
0  38-20, Review Avenue, Blissville, Queens, Quee...  
1  55, Wall Street, Whitehall, Manhattan Communit...  


In [3]:
df['pickup_location']

0    Naturalizer, 7, West 34th Street, Midtown Sout...
1    New York City Hall, 260, Broadway, Lower Manha...
Name: pickup_location, dtype: object

In [4]:
uber_data = pd.read_csv('uber_updated.csv')
uber_data.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,...,pickup_hour,pickup_minute,pickup_day_of_week,pickup_time_of_day,pickup_datetime_naive,SNOW,SNWD,TMAX,TMIN,is_holiday
0,55072468,2009-01-01 01:15:22.0000006,8.5,2009-01-01 01:15:22+00:00,-73.981918,40.779456,-73.957685,40.771043,2,2009,...,1,15,3,0,2009-01-01,0,0,-3.3,-9.4,1
1,30813112,2009-01-01 01:59:17.0000001,13.0,2009-01-01 01:59:17+00:00,-73.983759,40.721389,-73.994833,40.687179,2,2009,...,1,59,3,0,2009-01-01,0,0,-3.3,-9.4,1
2,8688426,2009-01-01 02:05:03.0000003,10.6,2009-01-01 02:05:03+00:00,-73.956635,40.771254,-73.991528,40.749778,2,2009,...,2,5,3,0,2009-01-01,0,0,-3.3,-9.4,1
3,13190369,2009-01-01 02:09:13.0000003,12.2,2009-01-01 02:09:13+00:00,-73.984605,40.728020,-73.955746,40.776830,1,2009,...,2,9,3,0,2009-01-01,0,0,-3.3,-9.4,1
4,45716268,2009-01-01 02:13:41.0000001,11.0,2009-01-01 02:13:41+00:00,-73.980127,40.737425,-74.009544,40.726025,4,2009,...,2,13,3,0,2009-01-01,0,0,-3.3,-9.4,1


In [5]:
import time

pickup_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}
dropoff_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}


t1 = time.time()
# Iterate over the rows of the DataFrame
for index, row in uber_data.tail(5).iterrows():

    pickup_latitude = row['pickup_latitude']
    pickup_longitude = row['pickup_longitude']
    pickup_location = reverse_geocode(pickup_latitude, pickup_longitude)
    pickup_location_list = location.split(', ')

    dropoff_latitude = row['dropoff_latitude']
    dropoff_longitude = row['dropoff_longitude']
    dropoff_location = reverse_geocode(dropoff_latitude, dropoff_longitude)
    dropoff_location_list = location.split(', ')

    pickup_boroughs[pickup_location_list[-6]] += 1
    dropoff_boroughs[dropoff_location_list[-6]] += 1 
    
    # print(location_list[-6])
    # print(len(location_list))
    print(pickup_location_list[-6])
    print(dropoff_location_list[-6])
    print('**************************************')
t2 = time.time()
print(t2-t1)

Manhattan
Manhattan
**************************************
Manhattan
Manhattan
**************************************
Manhattan
Manhattan
**************************************
Manhattan
Manhattan
**************************************
Manhattan
Manhattan
**************************************
5.162086248397827


In [6]:
pickup_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}
dropoff_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}

In [7]:
uber_data.shape

(199494, 22)

In [8]:
import geopandas as gpd
from shapely.geometry import Point

# Step 1: Load borough boundaries
# Assuming you have a GeoJSON file with the boundaries of NYC boroughs
boroughs = gpd.read_file('new-york-city-boroughs.geojson')

# Step 2: Convert latitude and longitude to a Shapely Point
latitude = 40.824312  # Example latitude
longitude = -73.926457  # Example longitude
location_point = Point(longitude, latitude)

# Step 3: Check which borough contains the point
borough_name = None
for _, borough in boroughs.iterrows():
    if borough['geometry'].contains(location_point):
        borough_name = borough['name']  # Assuming the borough name is in a column called 'borough'
        break

# Step 4: Output the result
if borough_name:
    print(f"The location is in the borough: {borough_name}")
else:
    print("The location is not in any recognized borough.")


DataSourceError: new-york-city-boroughs.geojson: No such file or directory

In [ ]:
# import time
# from borough_finder import BoroughFinder
# # Initialize the BoroughFinder with the path to your GeoJSON file
# geojson_path = 'new-york-city-boroughs.geojson'  # Adjust the path if necessary
# finder = BoroughFinder(geojson_path)


# pickup_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}
# dropoff_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}

# errors = 0
# counter = 0

# t1 = time.time()
# # Iterate over the rows of the DataFrame
# for index, row in uber_data.iterrows():
#     counter += 1
#     pickup_latitude = row['pickup_latitude']
#     pickup_longitude = row['pickup_longitude']
#     # pickup_location = reverse_geocode(pickup_latitude, pickup_longitude)
#     # pickup_location_list = location.split(', ')

#     pickup_borough = finder.find_borough(pickup_longitude, pickup_latitude)

#     dropoff_latitude = row['dropoff_latitude']
#     dropoff_longitude = row['dropoff_longitude']
#     # dropoff_location = reverse_geocode(dropoff_latitude, dropoff_longitude)
#     # dropoff_location_list = location.split(', ')

#     dropoff_borough = finder.find_borough(dropoff_longitude, dropoff_latitude)

#     try:
#         pickup_boroughs[pickup_borough] += 1
#         dropoff_boroughs[pickup_borough] += 1
#     except:
#         errors += 1
#         print(f'Error with pickup: {pickup_borough} and dropoff: {dropoff_borough}')
#         print(f'Current count: {errors} out of {counter}')

    
#     # print(location_list[-6])
#     # print(len(location_list))
#     # print(pickup_location_list[-6])
#     # print(dropoff_location_list[-6])
#     # print('**************************************')
# t2 = time.time()
# print(t2-t1)

In [9]:
print(pickup_boroughs)
print(dropoff_boroughs)

{'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}
{'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}


In [10]:
import time
from location_finder import LocationFinder
# Initialize the BoroughFinder with the path to your GeoJSON file

finder = LocationFinder()


# pickup_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}
# dropoff_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}

errors = 0
counter = 0

t1 = time.time()
# Iterate over the rows of the DataFrame
for index, row in uber_data.head(20).iterrows():
    counter += 1
    pickup_latitude = row['pickup_latitude']
    pickup_longitude = row['pickup_longitude']
    # pickup_location = reverse_geocode(pickup_latitude, pickup_longitude)
    # pickup_location_list = location.split(', ')

    pickup_loc = finder.find_location(pickup_longitude, pickup_latitude)

    dropoff_latitude = row['dropoff_latitude']
    dropoff_longitude = row['dropoff_longitude']
    # dropoff_location = reverse_geocode(dropoff_latitude, dropoff_longitude)
    # dropoff_location_list = location.split(', ')

    dropoff_loc = finder.find_location(dropoff_longitude, dropoff_latitude)

    try:
        # pickup_boroughs[pickup_borough] += 1
        # dropoff_boroughs[pickup_borough] += 1
        print(f'Pickup: {pickup_loc}')
        print(f'Dropoff: {dropoff_loc}')
    except:
        errors += 1

    # print(location_list[-6])
    # print(len(location_list))
    # print(pickup_location_list[-6])
    # print(dropoff_location_list[-6])
    print('**************************************')
t2 = time.time()
print(t2-t1)

Pickup: ('Upper West Side', 'Manhattan')
Dropoff: ('Lenox Hill', 'Manhattan')
**************************************
Pickup: ('Lower East Side', 'Manhattan')
Dropoff: ('Cobble Hill', 'Brooklyn')
**************************************
Pickup: ('Lenox Hill', 'Manhattan')
Dropoff: ('Chelsea', 'Manhattan')
**************************************
Pickup: ('Manhattan Community Board 3', 'Manhattan')
Dropoff: ('Upper East Side', 'Manhattan')
**************************************
Pickup: ('Manhattan Community Board 6', 'Manhattan')
Dropoff: ('Manhattan Community Board 2', 'Manhattan')
**************************************
Pickup: ('Manhattan Community Board 6', 'Manhattan')
Dropoff: ('Manhattan Community Board 3', 'Manhattan')
**************************************
Pickup: ('Manhattan Community Board 6', 'Manhattan')
Dropoff: ('Manhattan Community Board 6', 'Manhattan')
**************************************
Pickup: ('Lenox Hill', 'Manhattan')
Dropoff: ('Manhattan Community Board 1', 'Manhatt

In [11]:
import time
from borough_finder import BoroughFinder
# Initialize the BoroughFinder with the path to your GeoJSON file
geojson_path = 'nyc-neighborhoods-2.geojson'  # Adjust the path if necessary
finder = BoroughFinder(geojson_path)


# pickup_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}
# dropoff_boroughs = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 0}

errors = 0
counter = 0

t1 = time.time()
# Iterate over the rows of the DataFrame
for index, row in uber_data.head(20).iterrows():
    counter += 1
    pickup_latitude = row['pickup_latitude']
    pickup_longitude = row['pickup_longitude']
    # pickup_location = reverse_geocode(pickup_latitude, pickup_longitude)
    # pickup_location_list = location.split(', ')

    pickup_loc = finder.find_location(pickup_longitude, pickup_latitude)

    dropoff_latitude = row['dropoff_latitude']
    dropoff_longitude = row['dropoff_longitude']
    # dropoff_location = reverse_geocode(dropoff_latitude, dropoff_longitude)
    # dropoff_location_list = location.split(', ')

    dropoff_loc = finder.find_location(dropoff_longitude, dropoff_latitude)

    # try:
    #     pickup_boroughs[pickup_borough] += 1
    #     dropoff_boroughs[pickup_borough] += 1
    # except:
    #     errors += 1
    #     print(f'Error with pickup: {pickup_borough} and dropoff: {dropoff_borough}')
    #     print(f'Current count: {errors} out of {counter}')
    print(f'Pickup: {pickup_loc}')
    print(f'Dropoff: {dropoff_loc}')

    
    # print(location_list[-6])
    # print(len(location_list))
    # print(pickup_location_list[-6])
    # print(dropoff_location_list[-6])
    # print('**************************************')
t2 = time.time()
print(t2-t1)

Pickup: ('Upper West Side', 'Manhattan')
Dropoff: ('Upper East Side', 'Manhattan')
Pickup: ('Lower East Side', 'Manhattan')
Dropoff: ('Cobble Hill', 'Brooklyn')
Pickup: ('Upper East Side', 'Manhattan')
Dropoff: ('Chelsea', 'Manhattan')
Pickup: ('East Village', 'Manhattan')
Dropoff: ('Upper East Side', 'Manhattan')
Pickup: ('Gramercy', 'Manhattan')
Dropoff: ('SoHo', 'Manhattan')
Pickup: ('Gramercy', 'Manhattan')
Dropoff: ('East Village', 'Manhattan')
Pickup: ('Murray Hill', 'Manhattan')
Dropoff: ('Kips Bay', 'Manhattan')
Pickup: ('Upper East Side', 'Manhattan')
Dropoff: ('Financial District', 'Manhattan')
Pickup: ('Midtown', 'Manhattan')
Dropoff: ('Murray Hill', 'Manhattan')
Pickup: ('East Village', 'Manhattan')
Dropoff: ('Murray Hill', 'Manhattan')
Pickup: ('East Village', 'Manhattan')
Dropoff: ('Chelsea', 'Manhattan')
Pickup: ('Harlem', 'Manhattan')
Dropoff: ('Stuyvesant Town', 'Manhattan')
Pickup: ('Bath Beach', 'Brooklyn')
Dropoff: ('Sheepshead Bay', 'Brooklyn')
Pickup: ('Upper West

In [ ]:
# # Parallel Processing
# import concurrent.futures

# # Function to process each row of the DataFrame
# def process_row(row):
#     pickup_latitude = row['pickup_latitude']
#     pickup_longitude = row['pickup_longitude']
#     dropoff_latitude = row['dropoff_latitude']
#     dropoff_longitude = row['dropoff_longitude']

#     pickup_loc = finder.find_location(pickup_longitude, pickup_latitude)
#     dropoff_loc = finder.find_location(dropoff_longitude, dropoff_latitude)

#     return pickup_loc, dropoff_loc

# # Use ThreadPoolExecutor to process rows in parallel
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # Submit tasks to the executor
#     futures = {executor.submit(process_row, row): index for index, row in uber_data.head(20).iterrows()}
    
#     # Process the results as they complete
#     for future in concurrent.futures.as_completed(futures):
#         index = futures[future]
#         try:
#             pickup_loc, dropoff_loc = future.result()
#             print(f'Pickup: {pickup_loc}')
#             print(f'Dropoff: {dropoff_loc}')
#         except Exception as e:
#             errors += 1
#             print(f"Error processing row {index}: {e}")
#         finally:
#             print('**************************************')


In [ ]:
geojson_path = 'nyc-neighborhoods-2.geojson'  # Adjust the path if necessary
finder = BoroughFinder(geojson_path)

t1 = time.time()
uber_data['pickup_loc'] = uber_data.head(40).apply(lambda row: finder.find_location(row['pickup_longitude'], row['pickup_latitude']), axis=1)
uber_data['dropoff_loc'] = uber_data.head(40).apply(lambda row: finder.find_location(row['dropoff_longitude'], row['dropoff_latitude']), axis=1)
t2 = time.time()

print(t2-t1)

In [ ]:
uber_data.head(40)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Step 1: Load borough boundaries
# Assuming you have a GeoJSON file with the boundaries of NYC boroughs
boroughs = gpd.read_file('nyc-neighborhoods.geojson')

# Step 2: Convert latitude and longitude to a Shapely Point
latitude = 40.824312  # Example latitude
longitude = -73.926457  # Example longitude
location_point = Point(longitude, latitude)

# Step 3: Check which borough contains the point
borough_name = None
for _, borough in boroughs.iterrows():
    if borough['geometry'].contains(location_point):
        borough_name = borough['name']  # Assuming the borough name is in a column called 'borough'
        break

# Step 4: Output the result
if borough_name:
    print(f"The location is in the borough: {borough_name}")
else:
    print("The location is not in any recognized borough.")

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.validation import make_valid

# Step 1: Load borough boundaries
boroughs = gpd.read_file('nyc-neighborhoods-2.geojson')

# Step 2: Validate and fix geometries
def fix_geometry(geom):
    if not geom.is_valid:
        geom = make_valid(geom)  # Attempt to make the geometry valid
    if not geom.is_valid:  # Further attempt to fix with buffer(0)
        geom = geom.buffer(0)
    return geom

boroughs['geometry'] = boroughs['geometry'].apply(fix_geometry)

# Check if there are still any invalid geometries
invalid_geometries = boroughs[~boroughs['geometry'].is_valid]
if not invalid_geometries.empty:
    print("There are still invalid geometries:")
    print(invalid_geometries)

# Step 3: Convert latitude and longitude to a Shapely Point
latitude = 40.824312  # Example latitude
longitude = -73.926457  # Example longitude
location_point = Point(longitude, latitude)

# Step 4: Check which borough contains the point using 'within'
borough_name = None
for _, borough in boroughs.iterrows():
    if location_point.within(borough['geometry']):
        borough_name = borough['borough']  # Assuming the borough name is in a column called 'name'
        neighborhood_name = borough['neighborhood'] # Assuming the neighborhood name is in a column called 'neighborhood'
        break

# Step 5: Output the result
if borough_name and neighborhood_name:
    print(f"The location is in the borough: {borough_name} and neighborhood: {neighborhood_name}")
else:
    print("The location is not in any recognized borough.")


In [15]:
new_df = pd.read_csv('updated_uber_loc.csv')
new_df.head()
print(f'Before: {new_df.shape}')
new_df = new_df[(new_df['pickup_latitude'] != 0) & (new_df['pickup_longitude'] != 0) & (new_df['dropoff_latitude'] != 0) & (new_df['dropoff_longitude'] != 0)]
print(f'After: {new_df.shape}')

Before: (199494, 24)
After: (195545, 24)


0           ('Upper West Side', 'Manhattan')
1           ('Lower East Side', 'Manhattan')
2           ('Upper East Side', 'Manhattan')
3              ('East Village', 'Manhattan')
4                  ('Gramercy', 'Manhattan')
                         ...                
199489      ('Upper West Side', 'Manhattan')
199490    ('Greenwich Village', 'Manhattan')
199491         ('West Village', 'Manhattan')
199492     ('Theater District', 'Manhattan')
199493              ('Midtown', 'Manhattan')
Name: pickup_loc, Length: 195545, dtype: object

In [26]:
import ast

for index, row in uber_data.head(20).iterrows():
    pickup_loc_str = row[uber_data.columns.get_loc('pickup_loc')]
    pickup_loc_tuple = ast.literal_eval(pickup_loc_str)
    print(pickup_loc_tuple[0])


Upper West Side
Lower East Side
Upper East Side
East Village
Gramercy
Gramercy
Murray Hill
Upper East Side
Midtown
East Village
East Village
Harlem
Bath Beach
Upper West Side
Midtown
Flatiron District
Upper West Side
Battery Park City
Financial District
Harlem


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_14268\1801064162.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pickup_loc_str = row[uber_data.columns.get_loc('pickup_loc')]


In [27]:
import ast

# Convert the string representation of tuples to actual tuples and create new columns
uber_data['pickup_neighborhood'] = uber_data['pickup_loc'].apply(lambda x: ast.literal_eval(x)[0])
uber_data['pickup_borough'] = uber_data['pickup_loc'].apply(lambda x: ast.literal_eval(x)[1])
uber_data['dropoff_neighborhood'] = uber_data['dropoff_loc'].apply(lambda x: ast.literal_eval(x)[0])
uber_data['dropoff_borough'] = uber_data['dropoff_loc'].apply(lambda x: ast.literal_eval(x)[1])

# Delete the original columns
uber_data.drop(columns=['pickup_loc', 'dropoff_loc'], inplace=True)

# Display the first few rows to verify the changes
print(uber_data.head())

ValueError: malformed node or string: nan